# exploration and testing

In [ ]:
import pickle
pickle.dump(var_name, open(file_name, r/w/etc))
var_name = pickle.load(open(file_name, r/w/etc))

In [1]:
from bs4 import BeautifulSoup

In [21]:
with open ('afe199405', 'r') as infile:
    soup = BeautifulSoup(infile, 'lxml')

In [54]:
#not all elements are news stories with headlines etc.
#finds all elements with attribute type = 'story' and creates a list with one document per list item
elements = soup.find_all(type = 'story')

In [135]:
def dict_builder(soup_object):

    news_dict={}

    elements = soup.find_all(type = 'story')

    for document in elements:
        #we'll use the document identifier for index in datafram
        index = document['id']
        #the date may be useful in future, not all stories have dates
        if document.dateline:
            clean_date = document.dateline.text.strip()
        else:
            clean_date = 'None'
        #the headline will be part of the vector (not all stories have headlines)    
        if document.headline:
            clean_headline = document.headline.text.strip()
        else:
            clean_headline = 'None'
        # the first five sentences will be the other part of the vector - the limit=5 argument only returns 5
        #all linebreaks are replaced with spaces
        first_five = [el.text.strip().replace('\n', ' ') for el in document.find_all('p', limit=5)]
        #the dict is populated with the id as key and a tuple of date, headline and first five sentences
        news_dict[index]=(clean_date, clean_headline, first_five) 

    return news_dict

In [122]:
#inspect dictionary output here
# news_dict

In [123]:
# Dataframe construction - orient = index means that the key of the dict is the index of the df

import pandas as pd

columns=['date', 'headline', 'first_five']
df = pd.DataFrame.from_dict(news_dict, orient= 'index', columns = columns)
df

,date,headline,first_five
AFE19940512.0003,"UNDATED, May 12 (AFP)",Tributes pour in for late British Labour Party...,[Tributes poured in from around the world Thur...
AFE19940512.0004,"WASHINGTON, May 12 (AFP)",France rules out participation in military inv...,[France would not join a US military invasion ...
AFE19940512.0005,"WASHINGTON, May 12 (AFP)",Chinese dissidents in US favor partial revokin...,[Chinese dissidents in the United States gener...
AFE19940512.0006,"YEREVAN, May 12 (AFP)",Nagorno Karabakh hit by further clashes amid c...,[The Azerbaijani enclave of Nagorno Karabakh w...
AFE19940512.0008,"INDIANAPOLIS, Indiana, May 12 (AFP)",This restart looks good by Jim Slater,"[This time, Nigel Mansell will not be left beh..."
...,...,...,...
AFE19940531.0385,"UNITED NATIONS, May 31 (AFP)",Security Council agrees on draft resolution ur...,[The Security Council agreed late Tuesday on t...
AFE19940531.0386,"SANAA, June 1 (AFP)",War-torn Yemen hit by acute shortages by Laure...,[The conflict in Yemen has created an acute sh...
AFE19940531.0387,"VIENNA, June 1 (AFP)",North Korea ready to set aside 40 fuel rods fo...,[North Korea is prepared to store 40 fuel rods...
AFE19940531.0389,"CAEN, France, June 1 (AFP)","Authorities mount enormous security, logistics...",[French authorities are mounting an enormous l...


# working function

In [1]:
def dict_builder(soup_object):

    news_dict={}

    elements = soup.find_all(type = 'story')

    for document in elements:
        #we'll use the document identifier for index in datafram
        index = document['id']
        #the date may be useful in future, not all stories have dates
        if document.dateline:
            clean_date = document.dateline.text.strip()
        else:
            clean_date = 'None'
        #the headline will be part of the vector (not all stories have headlines)    
        if document.headline:
            clean_headline = document.headline.text.strip()
        else:
            clean_headline = 'None'
        # the first five sentences will be the other part of the vector - the limit=5 argument only returns 5
        #all linebreaks are replaced with spaces
        first_five = [el.text.strip().replace('\n', ' ') for el in document.find_all('p', limit=5)]
        #the dict is populated with the id as key and a tuple of date, headline and first five sentences
        news_dict[index]=(clean_date, clean_headline, first_five) 

    return news_dict

# En masse

- cycle through one large dataset (afp_eng; gzipped files)

- extract: name, date, headline, first five sentences as dictionary

- pickle


In [2]:
import gzip
import glob
import os
import pickle
from bs4 import BeautifulSoup

In [7]:
list_o_dicts = []
for file in glob.glob('GIGAword/giga.tar/gigaword/gigaword_eng_5_d1/data/afp_eng/*'):

    with gzip.open(file, 'r') as infile:
        basename = os.path.basename(file)
        soup = BeautifulSoup(infile, 'lxml')
        list_o_dicts.append(dict_builder(soup))
        pickle.dump(dict_builder(soup), open( f'./pickles/{basename}.pkl', "wb" ))
        print(basename)

afp_eng_200609.gz
afp_eng_200610.gz


KeyboardInterrupt: 

# extract pickles to large dataframe

In [1]:
import pickle
import glob
import pandas as pd

df_list=[]

for file in glob.glob('pickles/*'):
    pickle_dict = pickle.load(open(file, 'rb'))
    columns=['date', 'headline', 'first_five']
    df_list.append(pd.DataFrame.from_dict(pickle_dict, orient='index', columns = columns))
    


In [2]:
big_df = pd.concat(df_list)

In [5]:
# pd.set_options('display.max_rows', None)

big_df

,date,headline,first_five,related
AFP_ENG_19940512.0003,"UNDATED, May 12 (AFP)",Tributes pour in for late British Labour Party...,[Tributes poured in from around the world Thur...,NaN
AFP_ENG_19940512.0004,"WASHINGTON, May 12 (AFP)",France rules out participation in military inv...,[France would not join a US military invasion ...,NaN
AFP_ENG_19940512.0005,"WASHINGTON, May 12 (AFP)",Chinese dissidents in US favor partial revokin...,[Chinese dissidents in the United States gener...,NaN
AFP_ENG_19940512.0006,"YEREVAN, May 12 (AFP)",Nagorno Karabakh hit by further clashes amid c...,[The Azerbaijani enclave of Nagorno Karabakh w...,NaN
AFP_ENG_19940512.0008,"INDIANAPOLIS, Indiana, May 12 (AFP)",This restart looks good by Jim Slater,"[This time, Nigel Mansell will not be left beh...",NaN
...,...,...,...,...
AFP_ENG_20061031.0735,"BEIJING, Nov 1, 2006","Secret talks, discreet brokering ends NKorea d...",[One year of diplomatic deadlock on the North...,NaN
AFP_ENG_20061031.0736,"JAKARTA, Nov 1, 2006",Indonesia nabs couriers for top terrorist fugi...,[Indonesia's anti-terrorist police arrested t...,NaN
AFP_ENG_20061031.0737,"HONG KONG, Nov 1, 2006",Hong Kong shares end morning higher on fund in...,[Hong Kong share prices finished the morning ...,NaN
AFP_ENG_20061031.0738,"KUALA LUMPUR, Nov 1, 2006","Piracy on decline, but Bangladesh a hotspot: IMB","[Piracy declined globally this year, but Chit...",NaN


# test - keyword search - update 'related' column

In [5]:
big_df['related'] = 0

In [12]:
# .loc is necessary otherwise error reads "'Series' objects are mutable, thus they cannot be hashed"
big_df.loc[big_df['first_five'].str.contains('combat') & big_df['headline'].str.contains('poverty'), 'related'] = 1

In [13]:
big_df[big_df['related']==1]

,date,headline,first_five,related
AFP_ENG_19940512.0003,"UNDATED, May 12 (AFP)",Tributes pour in for late British Labour Party...,[Tributes poured in from around the world Thur...,1.0


In [ ]:
big_df.loc[big_df['headline'].str.contains('Tributes') & big_df['headline'].str.contains('Labour'), 'related'] = 1